In [1]:
import pandas as pd
import numpy as np
import time,os,sys
from p_tqdm import p_map
from tqdm import tqdm
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt

import sqlite3
sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))

from const import DATASETS,SETTINGS

TYPE310 Output_Areas 2001
TYPE299 Output_Areas 2011
TYPE304 Lower_Layer_Super_Output_Areas 2001
TYPE298 Lower_Layer_Super_Output_Areas 2011
TYPE305 Middle_Layer_Super_Output_Areas 2001
TYPE297 Middle_Layer_Super_Output_Areas 2011


### Merge all centroids for relevant Layer. 

In [2]:
# def get_centroids(kind):
areatype = 'Output_Areas'    
if 'Areas' in sys.argv[1]:
    areatype = sys.argv[1]

__outdir__='flatdb/'+areatype
try:os.mkdir(__outdir__)
except:pass

areatype


'Output_Areas'

In [55]:
mapping = pd.read_csv(__outdir__+'/mapping.csv')

geomap = dict(zip(mapping.AREAID,mapping.AREACD))

# mapping.groupby('PARENTS AREAID AREACD'.split()).count().index
groups = mapping.groupby('PARENTS').apply(lambda x: list(x.AREAID))
groups

PARENTS
E06000001    [E00060255, E00060256, E00060257, E00060258, E...
E06000002    [E00060556, E00060557, E00060558, E00060559, E...
E06000003    [E00060999, E00061000, E00061001, E00061002, E...
E06000004    [E00061461, E00061462, E00061463, E00061464, E...
E06000005    [E00062043, E00062044, E00062045, E00062046, E...
                                   ...                        
W06000020    [W00007742, W00007743, W00007744, W00007745, W...
W06000021    [W00008044, W00008045, W00008046, W00008047, W...
W06000022    [W00008328, W00008329, W00008330, W00008331, W...
W06000023    [W00002268, W00002269, W00002270, W00002271, W...
W06000024    [W00006756, W00006757, W00006758, W00006759, W...
Length: 173, dtype: object

##  A function to get the individual dataframes

In [17]:
def process_df(url, src):
    '''Read each individaul query and return a dataframe'''
    # print(url)
    dummydf = pd.read_csv(url)
    dummydf.columns = 'GEOGRAPHY_CODE YEAR CELL OBS_VALUE'.split()

    if 'cellmaps' in src:
        cells = '0,'+','.join(src['cellnames'])
        cells = ','.join(src['cellnames'])
        for v, k in src['cellmaps'].items(): 
            for c in k: 
                cells = cells.replace(c,v)
        src['cellnames'] = ('Totals,'+cells).split(',')

    mapping = dict(zip(eval(src['cells']), src['cellnames']))
    dummydf['CELL'] = dummydf['CELL'].apply(lambda x: mapping[x])   # (rename,mapping = mapping)
    return dummydf



## Process each Year and concat

In [18]:
def fetch(dataset):
    '''Fetch the data from the source'''
    global SETTINGS
    time.sleep(np.random.rand()*10.)
    sources = dataset['sources']

    df = False
    
    for src in sources:
        src = src.copy()
        src['cells'] = ','.join(map(str, src['cells']))
        src['selection'] = ','.join(map(str, src['selection']))
        src['key'] = SETTINGS.conf['apikey']
        src['geocode'] = SETTINGS.geocode[areatype][src['year']]
        if not hasattr(src, 'query_misc'): src['query_misc'] = ''

        url = SETTINGS.conf['apiurl'].format(**src)
        if type(df) != bool: df = pd.concat([df, process_df(url, src)]) # df.append(process_df(url,src))
        else: df = process_df(url, src)
        # print(df.tail(2))

    # return df
    print('mapping')
    # subcategory = mapping
    df = df[['GEOGRAPHY_CODE', 'CELL', 'OBS_VALUE','YEAR']]
    df = df.sort_values('CELL')
    order = df.CELL.drop_duplicates().to_list()


    if 'Totals' not in order: 
        b = df.groupby('GEOGRAPHY_CODE').sum()
        b.CELL = 'Totals'
        df = pd.concat([df,b.reset_index()],axis=0)  
        order=['Totals']+order

    mapping = dict(zip(order, range(len(order))))
    
    # print(order)

    # datamong
#  drop novalues

    df = df[df.GEOGRAPHY_CODE.isin(geomap.keys())]
    df['GEOGRAPHY_CODE'] = df['GEOGRAPHY_CODE'].apply(lambda x: geomap[x]).astype(np.uint16)
    i = df.groupby(['GEOGRAPHY_CODE', 'CELL']).sum().astype(np.uint32).reset_index()
    i.CELL = [mapping[k] for k in i.CELL]
    i = i[i.OBS_VALUE > 0] # remove zero values

    # return i[['GEOGRAPHY_CODE', 'CELL', 'OBS_VALUE']]]
    

    return [dataset['code'], order , i[['GEOGRAPHY_CODE', 'CELL', 'OBS_VALUE']]]


In [19]:
SETTINGS.geocode[areatype]

{'2001': 'TYPE310', '2011': 'TYPE299'}

### Parallel Fetch 


In [20]:
results = []
for i in tqdm(DATASETS):
    try:results.append(fetch(i))
    except:results.append(fetch(i))


# print(results)
# a = fetch(DATASETS[4])

  0%|          | 0/7 [00:00<?, ?it/s]

mapping


 14%|█▍        | 1/7 [06:09<36:54, 369.05s/it]

mapping


 29%|██▊       | 2/7 [07:53<17:47, 213.49s/it]

...

 71%|███████▏  | 5/7 [15:49<05:40, 170.06s/it]

mapping


 86%|████████▌ | 6/7 [17:59<02:36, 156.28s/it]

mapping


100%|██████████| 7/7 [21:40<00:00, 185.82s/it]


In [22]:
results[0]

['population',
 ['Totals', 'female', 'male'],
         GEOGRAPHY_CODE  CELL  OBS_VALUE
 0                    0     1        556
 1                    0     2        578
 2                    1     1        530
 3                    1     2        574
 4                    2     1        208
 ...                ...   ...        ...
 130963           65533     2        311
 130964           65534     1        393
 130965           65534     2        358
 130966           65535     1        420
 130967           65535     2        389
 
 [130968 rows x 3 columns]]

In [36]:
cellmap  = {}
for name,columns,_ in results:
    cellmap[name] = dict(zip(range(len(columns)),columns))

cellmap[results[0][0]]

print('write cell maps to json')


write cell maps to json


In [38]:
# import pickle
# pickle.dump( results, open( "results.p", "wb" ) )
# pickle.load( open( "save.p", "rb" ) )

In [46]:
for p in mapping.PARENTS:
    df =  mapping.iloc[p]
df

TypeError: Cannot index by location index with a non-integer key

In [4]:


# cur = conn.cursor()


for name,order,df in results:
    print(name)
    order = pd.DataFrame(order, columns = ['NAME'])
    # order['CELL'] = order.index
    # print(order)
    df.to_sql(name, con=conn, if_exists='replace',index_label='GEOMETRY_ID')
    order.to_sql(name+'_names', con=conn, if_exists='replace',index_label='CELL')



NameError: name 'results' is not defined

### Write To DB

### SHRINK THE DATABASE

In [ ]:
from pathlib import Path
file_size = Path(db_file).stat().st_size

print("The file size WAS:", file_size/1048576,"MB")

conn.execute("VACUUM") ### SHRINK!!!!
conn.close()

file_size = Path(db_file).stat().st_size

print("The file size is:", file_size/1048576,"MB")

### CENTROIDS table

In [ ]:
# a = results[1]
# print(a)
# print(a[a.YEAR==2011])
# # [geomap[i] for i in a.GEOGRAPHY_CODE]

# geomap
# e=a[:].GEOGRAPHY_CODE.drop_duplicates()
# print(e)
# s = 0 
# f = 0
# miss = []
# for i in e.values:

#     try:
#         (geomap[i])
#         s+=1
#     except:
#         f+=1
#         miss.append(i)
# print(s,f,len(miss))
        
# print(miss[:20])
# geomap